<a href="https://colab.research.google.com/github/RalfH1388/genai-lecture/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from dotenv import load_dotenv


ModuleNotFoundError: No module named 'dotenv'